# AI YouTube Shorts Pipeline (Notebook)

This notebook automates: story generation (OpenAI), video generation (Pika/Runway), uploading to YouTube Shorts, and cleanup. It's designed to be run non-interactively (via GitHub Actions). Replace the environment variables in GitHub Secrets.

**Required secrets (set in GitHub repo Settings → Secrets):**
- `OPENAI_API_KEY`
- `PIKA_API_KEY` (or set `VIDEO_PROVIDER` to `runway` and provide `RUNWAY_API_KEY`)
- `YOUTUBE_CLIENT_ID`
- `YOUTUBE_CLIENT_SECRET`
- `YOUTUBE_REFRESH_TOKEN` (recommended)

Run this notebook locally first to verify before enabling Actions.

In [ ]:
!pip install python-dotenv openai google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client moviepy requests


In [ ]:
import os
import json
from dotenv import load_dotenv

# Load .env locally — GitHub Actions will ignore this and use ENV variables
if os.path.exists(".env"):
    load_dotenv()

print("Environment loaded successfully.")


In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PIKA_API_KEY = os.getenv("PIKA_API_KEY")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")

GOOGLE_CREDS = os.getenv("GOOGLE_CREDENTIALS")  # Full JSON string
GOOGLE_DATA = json.loads(GOOGLE_CREDS) if GOOGLE_CREDS else None

print("Keys loaded:", "OPENAI" if OPENAI_API_KEY else None,
      "PIKA" if PIKA_API_KEY else None,
      "ELEVEN" if ELEVENLABS_API_KEY else None,
      "GOOGLE" if GOOGLE_DATA else None)


In [ ]:
import openai

client = openai.OpenAI(api_key=OPENAI_API_KEY)

def generate_script(animal="baby penguin"):
    prompt = f"Write a 10-second cute scene involving a {animal}."

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    script = response.choices[0].message.content
    print("Script:", script)
    return script

script = generate_script()


In [ ]:
import requests
import time

def create_video_pika(prompt):
    url = "https://api.pika.art/v1/video"

    headers = {"Authorization": f"Bearer {PIKA_API_KEY}"}
    payload = {"prompt": prompt}

    res = requests.post(url, json=payload, headers=headers)
    video_id = res.json()["id"]

    print("Video ID:", video_id)

    # Poll until ready
    while True:
        status = requests.get(f"{url}/{video_id}", headers=headers).json()
        if status["status"] == "completed":
            video_url = status["video_url"]
            print("Video Ready:", video_url)
            break
        time.sleep(2)

    # Download video
    vid_file = "video.mp4"
    open(vid_file, "wb").write(requests.get(video_url).content)
    return vid_file

video_path = create_video_pika(script)
video_path


In [ ]:
import requests

def create_audio(text):
    url = "https://api.elevenlabs.io/v1/text-to-speech/EXAVITQu4vr4xnSDxMaL"  
    headers = {"xi-api-key": ELEVENLABS_API_KEY}
    payload = {"text": text, "voice_settings": {"stability": 0.3}}

    audio = requests.post(url, json=payload, headers=headers)
    path = "audio.mp3"
    open(path, "wb").write(audio.content)
    return path

audio_path = create_audio(script)
audio_path


In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip

output_file = "final_video.mp4"

video = VideoFileClip(video_path)
audio = AudioFileClip(audio_path)

final = video.set_audio(audio)
final.write_videofile(output_file)

output_file


In [ ]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

def upload_youtube(video_file, title="Cute Animal Short", desc="Generated by AI"):
    creds = Credentials.from_authorized_user_info(GOOGLE_DATA["installed"])

    youtube = build("youtube", "v3", credentials=creds)

    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {"title": title, "description": desc, "categoryId": "15"},
            "status": {"privacyStatus": "public"}
        },
        media_body=MediaFileUpload(video_file)
    )
    response = request.execute()
    print("Uploaded:", response["id"])
    return response["id"]

yt_id = upload_youtube("final_video.mp4")
yt_id


In [ ]:
print("Pipeline finished successfully!")
print("YouTube Video ID:", yt_id)
